In [7]:
import numpy as np

In [264]:
def T(d):
    return np.matrix([[1, d], [0, 1]])

pupil_height=12.3

T1 = T(7.700)

T2 = T(1.850)

T3 = T(3.520)

T4 = T(1.850)

T5 = T(4.180)

# now, we've reached the aperature plane
# height = 11.6
T6 = T(3.000)

T7 = T(1.850)

T8 = T(7.270)

T9 = T(83.91)
# 81.320
Ts = [T1, T2, T3, T4, T5, T6, T7, T8, T9]

In [265]:
red_refr = [1.652, 1.5991, 1, 1.6396, 1, 1, 1.5776, 1.68990, 1] # wavelength = 650  1.6664
green_refr = [1.652, 1.6113, 1, 1.65, 1, 1, 1.5885, 1.6999, 1] #510  1.6764
blue_refr = [1.652, 1.6164, 1, 1.6542, 1, 1, 1.5930, 1.7040, 1] #475 1.6804

curvatures = [1/30.810, 1/-89.350, 1/580.380, 1/-80.630, 1/28.340, 0, 0, 1/32.190, 1/-52.990, 1/81.320]

def generate_R(c, n1, n2):
    return np.matrix([[1, 0], [c*(n1-n2)/n2, n1/n2]])

def create_Rs_for_color(color_refr):
    R = []
    prev_n = 1.000
    for i in range(len(color_refr)):
        R.append(generate_R(curvatures[i], prev_n, color_refr[i]))
        prev_n = color_refr[i]
    return R

def generate_L(c):
    return np.matrix([[1, 0], [2*c, 1]])

def create_Ls():
    L = []
    for i in range(len(curvatures)):
        L.append(generate_L(curvatures[i]))
    return L
        
L = create_Ls()
    
R_red = creat_Rs_for_color(red_refr)
R_green = creat_Rs_for_color(green_refr)
R_blue = creat_Rs_for_color(blue_refr)

print(L[1])
print(np.matrix([[1, 0], [2/-89.350, 1]]))

# problems:Lakn7 - keep prev value

[[ 1.          0.        ]
 [-0.02238388  1.        ]]
[[ 1.          0.        ]
 [-0.02238388  1.        ]]


In [301]:
def trace_ray_auto(r, theta, i, j, color_refr):
    
    ray = np.array([r, theta]).reshape(1, -1).T
    
    i, j = min(i, j), max(i, j) 
    
    M = np.identity(2)
    
    # forward through j-1
    for k in range(j):
        M = Ts[k].dot(color_refr[k]).dot(M)
    
    # reflect off of Lj
    M = L[j].dot(M)
    
    # Inverse
    for k in range(i+1, j-2, -1):
        M = np.linalg.inv(color_refr[k]).dot(Ts[k]).dot(M)
    
    M = Ts[i].dot(np.linalg.inv(L[i])).dot(Ts[i]).dot(M)

    
    for k in range(i+1, 9):
        if k == 5:
            after_ap = M.dot(ray)
            if after_ap[0, 0] > 11.6 or after_ap[0, 0] < -11.6:
                # recast
                print("recasting, got aperature radius: ", after_ap[0, 0])
                
                
                r_a = 11.6
                if r<0:
                    r_a = -11.5
                r_e = (r_a - M[0, 1]*theta)/M[0, 0]
                print("ray we're casting: ", r_e, theta)
                # modify ray
                ray = np.array([r_e, theta]).reshape(1, -1).T
            # crossing the aperture!
            M = Ts[k].dot(M)
            continue
        M = Ts[k].dot(color_refr[k]).dot(M)
    
    return M.dot(ray)

In [302]:
def trace_ray(r, theta, R):
    r = np.array([r, theta])

    Ma = Ts[4].dot(R[4]).dot(Ts[3]).dot(R[3]).dot(Ts[2]).dot(R[2]).dot(Ts[1]).dot(R[1]).dot(Ts[0]).dot(R[0])
    Ms = Ts[8].dot(R[8]).dot(Ts[7]).dot(R[7]).dot(Ts[6]).dot(R[6]).dot(Ts[5])
    
#     print("trace_ray")
#     print(Ma)
#     print(Ms)
    
    return Ms.dot(Ma).dot(r)
    
def trace_ray_with_ghost(r, theta):
    
    L4 = L[3]
    L2 = L[1]
    R3_inv = np.linalg.inv(R3)
    L2_inv = np.linalg.inv(L2)
    
    M1 = T3.dot(R3).dot(T2).dot(R2).dot(T1).dot(R1)
    Bounce = T2.dot(L2_inv).dot(T2).dot(R3_inv).dot(T3).dot(L4)
    M2 = T5.dot(R5).dot(T4).dot(R4).dot(T3).dot(R3)
    Ms = T9.dot(R9).dot(T8).dot(R8).dot(T7).dot(R7).dot(T6)
    
    Ma = M2.dot(Bounce).dot(M1)
    
    
    # ray at aperature
    ray_ap = Ma.dot(np.array([r, theta]))
    print("ray before recast", ray_ap) # since it's a 1 by 2 matrix
    if ray_ap[0, 0] > 11.6 or ray_ap[0, 0] < -11.6:
        # recast
        print("recasting")
        theta_e = theta
        r_a = 11.6
        if r<0:
            r_a = -11.5
        r_e = (r_a - Ma.__getitem__((0,1))*theta_e)/Ma.__getitem__((0,0))
        ray = np.array([r_e, theta])
        print("ray we're casting", ray)
        return Ms.dot(Ma).dot(ray)
    return Ms.dot(Ma).dot(np.array([r, theta]))

In [304]:
print(trace_ray_auto(14.5, 0.025, 1, 3, R_red)) # -3.16-10.18 = -13.34
print(trace_ray_auto(14.5, 0.025, 1, 3, R_green)) #-2.41-9.45 = -11.6
print(trace_ray_auto(14.5, 0.025, 1, 3, R_blue))
print(trace_ray_auto(-14.5, 0.025, 1, 3, R_red))
print(trace_ray_auto(-14.5, 0.025, 1, 3, R_green))
print(trace_ray_auto(-14.5, 0.025, 1, 3, R_blue))
print(trace_ray_auto(-14.5, 0.025, 3, 1, R_blue))

[[-3.16489143]
 [-0.12955141]]
[[10.17540939]
 [ 0.18744361]]


In [306]:
trace_ray(15, 0, R_green) # this lines up at 0 radius # focused on the green wavelength now

matrix([[ 0.00069748, -0.14750162]])

matrix([[ 0.00069748, -0.14750162]])

ray before recast [[8.57311603 0.04778441]]
[[ 0.80637476 -0.09885542]]
ray before recast [[-5.77394628  0.13695084]]
[[12.91711268  0.21463147]]


11.932119722842128
recasting
ray we're casting [13.82867985  0.17      ]
[[17.55491082  0.04733274]]
-2.4149425790923083
[[29.38529762  0.35356272]]


In [73]:
# part 2

# given the two radii, determine the size
def size(r1, r2):
    diff = abs(r1-r2)
    return diff * diff

# given radiance of light going in to entrance pupil, the flare intensity is radiance in pupil * flare quad size / entrance pupil size
# size matters


In [74]:
size(24.78056262, 24.24677832)

0.2849256789264907

In [75]:
# sensor plane size
12.3 *12.3

151.29000000000002

In [307]:
# changing radius should ghost linearly
print(trace_ray_with_ghost(0.5, 0.001))
print(trace_ray_with_ghost(1, 0.001))

ray before recast [[0.27535484 0.00031   ]]
[[-0.08257116 -0.00424719]]
ray before recast [[ 0.52271798 -0.00122735]]
[[-0.3053758  -0.00965213]]


In [308]:
# different ghosts should have different sizes


ray before recast [[ 6.11312502 -0.03597151]]
[[-5.3407606  -0.13180392]]


TypeError: 'list' object is not callable

In [178]:
# changing theta
print(trace_ray_with_ghost_2(12.3, 0.01))
print(trace_ray_with_ghost_2(12.3, 0.02))
print(trace_ray_with_ghost_2(12.3, 0.03))


# radius always gets bigger with bigger theta... 
# so if the sun is further away from the center, there's bigger lens flares? 
# not sure this is true

[[-32300.86635261   -305.01473834]]
[[-32634.05130402   -308.1609856 ]]
[[-32967.23625542   -311.30723285]]
